In [1]:
!pip install --upgrade huggingface_hub peft evaluate

import huggingface_hub
import peft

print(huggingface_hub.__version__)
print(peft.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
0.27.1
0.14.0


In [4]:
# Import wandb library
import wandb

# Log in to wandb using your API key
wandb.login(key='5656979c25c03784238533b8e79512ab61bf94e2')

print("Logged in to wandb successfully.")

wandb: Currently logged in as: hadiabbas223 (hadiabbas223-iba-institute-of-business-administration). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Logged in to wandb successfully.


# 1. Import Libraries


In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset

print("Libraries imported successfully.")


Libraries imported successfully.


# 2. Load Datasets



In [3]:
# Load datasets
train_data = pd.read_csv('/kaggle/input/imdb-dataset/train.csv')
test_data = pd.read_csv('/kaggle/input/imdb-dataset/test.csv')

print("Training Data Sample:")
print(train_data.head())
print("\nTesting Data Sample:")
print(test_data.head())


Training Data Sample:
                                              review sentiment
0  SAPS AT SEA <br /><br />Aspect ratio: 1.37:1<b...  negative
1  If you want mindless action, hot chicks and a ...  positive
2  "The Woman in Black" is easily one of the cree...  positive
3  I can barely find the words to describe how mu...  negative
4  What's in here ?! Let me tell you. It's the pr...  negative

Testing Data Sample:
                                              review sentiment
0  Steven Rea plays a forensic scientist thrust o...  positive
1  As the first of the TV specials offered on the...  positive
2  There may something poetically right in seeing...  negative
3  all i can say about this film is to read the b...  negative
4  I thought it was a pretty good movie and shoul...  positive


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Load your training and test data
train_data = pd.read_csv('/kaggle/input/imdb-dataset/train.csv')
test_data = pd.read_csv('/kaggle/input/imdb-dataset/test.csv')

# Preprocessing: Convert the sentiment into binary labels (positive=1, negative=0)
train_data['label'] = train_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
test_data['label'] = test_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

# Use only 10% of the data for quick training and testing
train_data_sample = train_data.sample(frac=0.1, random_state=42).reset_index(drop=True)
test_data_sample = test_data.sample(frac=0.1, random_state=42).reset_index(drop=True)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define the dataset for Hugging Face Trainer
train_dataset = Dataset.from_pandas(train_data_sample[['review', 'label']])
test_dataset = Dataset.from_pandas(test_data_sample[['review', 'label']])

# Tokenize the reviews
def tokenize_function(examples):
    return tokenizer(examples['review'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)




/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
# Define model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english', num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",     # evaluation is done at the end of each epoch
    save_strategy="epoch",           # save model at the end of each epoch
    disable_tqdm=False,
    load_best_model_at_end=True,     # load best model after training
)

# Trainer setup
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics=None,                # Leave metrics calculation separate
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

print(f"Evaluation Results: {results}")

# Predictions for classification report
predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(axis=-1)

# Print classification report
print(classification_report(test_data_sample['label'], y_pred))

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.346500,0.256328
2,0.303100,0.304224
3,0.106200,0.366728


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

Evaluation Results: {'eval_loss': 0.25632813572883606, 'eval_runtime': 20.0697, 'eval_samples_per_second': 99.653, 'eval_steps_per_second': 6.228, 'epoch': 3.0}
              precision    recall  f1-score   support

           0       0.93      0.86      0.89       966
           1       0.88      0.94      0.91      1034

    accuracy                           0.90      2000
   macro avg       0.90      0.90      0.90      2000
weighted avg       0.90      0.90      0.90      2000

